In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from load_data_and_distribution_analisys import split_dataset_v1, load_dataset_cleaned

/Users/marcodeano/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/marcodeano/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Esempio con Random Forest

Iniziamo ad utilizzare dei modelli un po' più complessi e poco alla volta andiamo a migliorare il nostro modello con tecniche di preprocessing e tuning dei parametri; partiamo da una RandomForest (da adesso in poi le predizioni e le metriche, verranno salvate in nomi di variabili che contengono il nome del modello e la lettera 's' per smoke e 'd' per drink).


In [2]:
dataset_cleaned = load_dataset_cleaned()

Path to dataset files: /Users/marcodeano/.cache/kagglehub/datasets/sooyoungher/smoking-drinking-dataset/versions/2
   sex  age  height  weight  waistline  sight_left  sight_right  hear_left  \
0    1   35     170      75       90.0         1.0          1.0        1.0   
1    1   30     180      80       89.0         0.9          1.2        1.0   
2    1   40     165      75       91.0         1.2          1.5        1.0   
3    1   50     175      80       91.0         1.5          1.2        1.0   
4    1   50     165      60       80.0         1.0          1.2        1.0   

   hear_right    SBP  ...  LDL_chole  triglyceride  hemoglobin  urine_protein  \
0         1.0  120.0  ...      126.0          92.0        17.1            1.0   
1         1.0  130.0  ...      148.0         121.0        15.8            1.0   
2         1.0  120.0  ...       74.0         104.0        15.8            1.0   
3         1.0  145.0  ...      104.0         106.0        17.6            1.0   
4         1

In [3]:
X_smoke, Y_smoke, X_drink, Y_drink = split_dataset_v1(dataset_cleaned)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_smoke, Y_smoke, test_size=0.3, random_state=42, stratify=Y_smoke)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_drink, Y_drink, test_size=0.3, random_state=42, stratify=Y_drink)

### Esempio smoke

In [4]:
forest = RandomForestClassifier(random_state=42, class_weight="balanced")
forest.fit(X_train_s, y_train_s)
y_pred_rf_s = forest.predict(X_test_s)
report = classification_report(y_test_s, y_pred_rf_s)
print(report)

              precision    recall  f1-score   support

         1.0       0.82      0.84      0.83    167528
         2.0       0.44      0.35      0.39     47377
         3.0       0.51      0.56      0.54     58574

    accuracy                           0.69    273479
   macro avg       0.59      0.58      0.58    273479
weighted avg       0.69      0.69      0.69    273479



In [5]:
forest = RandomForestClassifier(random_state=42)
forest.fit(X_train_d, y_train_d)
y_pred_rf_d = forest.predict(X_test_d)
report = classification_report(y_test_d, y_pred_rf_d)
print(report)

              precision    recall  f1-score   support

           0       0.72      0.71      0.72    135539
           1       0.72      0.73      0.73    137940

    accuracy                           0.72    273479
   macro avg       0.72      0.72      0.72    273479
weighted avg       0.72      0.72      0.72    273479



In [ ]:
# Procediamo con il selezionare un paio di parametri e combinarli tra loro per vedere quale modello ottiene i risultati migliori

parameters = {
    "n_estimators": [50, 100, 200],
    "max_depth": [5, 10, 20],
    "min_samples_split": [50, 100, 200]
}

In [ ]:
for n_estimators in parameters["n_estimators"]:
    for max_depth in parameters["max_depth"]:
        for min_samples_split in parameters["min_samples_split"]:
            forest = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, random_state=42, class_weight="balanced")
            forest.fit(X_train_s, y_train_s)
            y_pred_rf_s = forest.predict(X_test_s)
            accuracy_rf_s = accuracy_score(y_test_s, y_pred_rf_s)
            precision_rf_s = precision_score(y_test_s, y_pred_rf_s, average="weighted")
            recall_rf_s = recall_score(y_test_s, y_pred_rf_s, average="weighted")
            f1_rf_s = f1_score(y_test_s, y_pred_rf_s, average="weighted")
            print(f"n_estimators: {n_estimators}, max_depth: {max_depth}, min_samples_split: {min_samples_split}")
            print(f"    Accuratezza: {accuracy_rf_s*100:.2f}%")
            print(f"    Precisione: {precision_rf_s*100:.2f}%")
            print(f"    Recall: {recall_rf_s*100:.2f}%")
            print(f"    F1: {f1_rf_s*100:.2f}%")

La miglior scelta dei parametri risulta essere la seguente:
- **n_estimators**: 100
- **max_depth**: 20
- **min_samples_split**: 50

In [8]:
forest = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_split=50, random_state=42, class_weight="balanced")
forest.fit(X_train_s, y_train_s)
y_pred_rf_s = forest.predict(X_test_s)
report = classification_report(y_test_s, y_pred_rf_s)
print(report)

              precision    recall  f1-score   support

         1.0       0.94      0.73      0.82    179620
         2.0       0.42      0.58      0.49     52107
         3.0       0.49      0.64      0.55     63795

    accuracy                           0.68    295522
   macro avg       0.62      0.65      0.62    295522
weighted avg       0.75      0.68      0.70    295522



In [ ]:
for n_estimators in parameters["n_estimators"]:
    for max_depth in parameters["max_depth"]:
        for min_samples_split in parameters["min_samples_split"]:
            forest = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, random_state=42)
            forest.fit(X_train_d, y_train_d)
            y_pred_rf_d = forest.predict(X_test_d)
            accuracy_rf_d = accuracy_score(y_test_d, y_pred_rf_d)
            precision_rf_d = precision_score(y_test_d, y_pred_rf_d)
            recall_rf_d = recall_score(y_test_d, y_pred_rf_d)
            f1_rf_d = f1_score(y_test_d, y_pred_rf_d)
            print(f"n_estimators: {n_estimators}, max_depth: {max_depth}, min_samples_split: {min_samples_split}")
            print(f"    Accuratezza: {accuracy_rf_d*100:.2f}%")
            print(f"    Precisione: {precision_rf_d*100:.2f}%")
            print(f"    Recall: {recall_rf_d*100:.2f}%")
            print(f"    F1: {f1_rf_d*100:.2f}%")

La miglior scelta dei parametri risulta essere la seguente:
- **n_estimators**: 100
- **max_depth**: 20
- **min_samples_split**: 50

In [9]:
forest = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_split=50, random_state=42)
forest.fit(X_train_d, y_train_d)
y_pred_rf_d = forest.predict(X_test_d)
report = classification_report(y_test_d, y_pred_rf_d)
print(report)

              precision    recall  f1-score   support

           0       0.74      0.71      0.73    147594
           1       0.72      0.75      0.74    147928

    accuracy                           0.73    295522
   macro avg       0.73      0.73      0.73    295522
weighted avg       0.73      0.73      0.73    295522



Ora che sono stati definiti i parametri migliori per le random forest, procediamo con la tecnica della feature selection con la speranza di migliorare i risultati di accuratezza, precisione, recall e f1_score; pensiamo sia un'operazione da compiere per poter ottenere risultati migliori, perchè abbiamo a che fare con un dataset con 22 feature e il rischio di overfit c'è. Probabilmente con una random forest questo rischio è minore, però testiamo se anche con una random forest i risultati migliorano.